In [ ]:
%matplotlib notebook
%load_ext rpy2.ipython
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
pandas2ri.activate()
readRDS = robjects.r['readRDS']


def plot_benchmarks(value, num_nodes, ilp_solvers, **kwargs):
    ni = 3 if "num_inputs" not in kwargs else kwargs["num_inputs"]
    nm = 3 if "num_meas" not in kwargs else kwargs["num_meas"]
    
    get_benchmark = lambda x, y : pd.read_csv(f"Output/N{x}_I{ni}_M{nm}_S1_P0.2/{y}/benchmark.tsv", sep="\t")
    get_inter = lambda x : len(readRDS(f'Output/N{x}_I{ni}_M{nm}_S1_P0.2/carnival_input.Rds')[2])
    
    data = np.array([[get_benchmark(x, y)[value].values for x in num_nodes] for y in ilp_solvers])
    
    if "scale" in kwargs: data = kwargs["scale"](data)
    x_ticklabels = num_nodes
    x_label = "Number of nodes"
    if "interactions" in kwargs:
        x_ticklabels = [get_inter(x) for x in num_nodes]
        x_label = "Number of interactions"
        
    fig, axs = plt.subplots(1, len(ilp_solvers), figsize=(12, 5), sharey=True)
    for i, ax in enumerate(axs):
        ax.boxplot(data[i].tolist())
        ax.set_xticklabels(x_ticklabels)
        ax.set_xlabel(x_label)
        ax.set_title(ilp_solvers[i])
        
    if "ylabel" in kwargs:
        axs[0].set_ylabel(kwargs["ylabel"])

        
t_args1 = {"num_nodes": np.arange(10, 101, 10), 
          "ilp_solvers": ["cplex", "cbc", "lpSolve"], 
          "ylabel": "Execution time [mins]", 
          "scale": lambda x : x/60}
mem_args1 =  {"num_nodes": np.arange(10, 101, 10), 
              "ilp_solvers": ["cplex", "cbc", "lpSolve"], 
              "ylabel": "Memory [GB]", 
              "scale": lambda x : x/1024}

t_args2 = {"num_nodes": np.arange(100, 350, 50), 
          "ilp_solvers": ["cplex", "cbc"], 
          "ylabel": "Execution time [mins]", 
          "scale": lambda x : x/60}
mem_args2 =  {"num_nodes": np.arange(100, 350, 50), 
              "ilp_solvers": ["cplex", "cbc"], 
              "ylabel": "Memory [GB]", 
              "scale": lambda x : x/1024}

# Smaller networks

In [ ]:
plot_benchmarks("s", **t_args1)

In [ ]:
plot_benchmarks("max_rss", nodes, solvers, **mem_args)

# Medium networks

In [ ]:
plot_benchmarks("s", np.arange(100, 350, 50), ["cplex", "cbc"], **t_args)

In [ ]:
plot_benchmarks("s", np.arange(100, 350, 50), ["cplex", "cbc"], **t_args, interactions=True)

In [ ]:
plot_benchmarks("max_rss", np.arange(100, 350, 50), ["cplex", "cbc"], **mem_args)

In [ ]:
plot_benchmarks("max_rss", np.arange(100, 350, 50), ["cplex", "cbc"], **mem_args, interactions=True)

In [ ]:
%%R
print(readRDS("Output/N100_I3_M3_S1_P0.2/cplex/result.Rds")$weightedSIF)
print(readRDS("Output/N100_I3_M3_S1_P0.2/cbc/result.Rds")$weightedSIF)
print(readRDS("Output/N100_I3_M3_S1_P0.2/lpSolve/result.Rds")$weightedSIF)